In [1]:
torch.setdefaulttensortype('torch.FloatTensor')

## Integral image using CPU

In [2]:
collectgarbage()

img = torch.FloatTensor(200, 256, 512)
torch.manualSeed(666)
img:randn(img:size())
result = torch.DoubleTensor(img:size(1), img:size(2)+1, img:size(3)+1)

cv = require 'cv'
require 'cv.imgproc'

times = torch.Tensor(10)

for k = 1,times:nElement() do
    local timer = torch.Timer()
    
    for _ = 1,10 do
        for i = 1,img:size(1) do
            cv.integral{img[i], result[i]}
        end
    end
    
    times[k] = timer:time().real
    io.stdout:write(k .. ' ')
    io.stdout:flush()
end

print()
print(times)
print('Average: ' .. times:mean() .. ' +/- ' .. times:std() * 3 .. ' seconds')

1 

2 

3 

4 

5 

6 

7 

8 

9 

10 
 0.7331
 0.5706
 0.5593
 0.5503
 0.5551
 0.6399
 0.6080
 0.5516
 0.5545
 0.5561
[torch.FloatTensor of size 10]

Average: 0.58784213066101 +/- 0.17674524630543 seconds	


## Integral image using GPU

In [3]:
require 'cutorch'
require 'IntegralSmartNorm'

collectgarbage()

imgGPU = img:cuda() --torch.CudaTensor(200, 256, 512)
-- torch.manualSeed(666)
-- cutorch.manualSeed(666)
-- imgGPU:randn(imgGPU:size())
resultGPU = torch.CudaTensor(imgGPU:size(1), imgGPU:size(2)+1, imgGPU:size(3)+1)
tmp = resultGPU:clone()

times = torch.Tensor(10)

for k = 1,times:nElement() do
    local timer = torch.Timer()
    
    for _ = 1,10 do
        integralCUDA(imgGPU, resultGPU, tmp)
    end
    
    cutorch.synchronize()
    times[k] = timer:time().real
    io.stdout:write(k .. ' ')
    io.stdout:flush()
end

print(times)
print('Average: ' .. times:mean() .. ' +/- ' .. times:std() * 3 .. ' seconds')

1 

2 

3 

4 

5 

...

7 

8 

9 

10 

0.01 *
 4.9829
 4.9315
 5.0465
 4.9628
 4.8538
 4.8126
 4.8576
 4.7347
 4.7511
 4.7983
[torch.FloatTensor of size 10]

Average: 0.048731803894043 +/- 0.0031221635827591 seconds	


In [ ]:
rows: 0.32688353061676
cols: 0.01458089351654
total: 
(sum is 0.3414644241333)

transpose:         0.00891916751861

using inplace transpose: 0.33936667442322
using inplace "skinny" transpose: 0.1487481355667
using out-of-place transpose: 0.048898983001709

In [1]:
require 'cutorch'
require 'IntegralSmartNorm'

t = torch.CudaTensor{{1,2,3,4}, {5,6,7,8}, {9,10,11,12}}:view(1, 3, 4):repeatTensor(2, 1, 1)
imgGPU = t
imgGPUorig = t:clone()
print(t)
res = torch.CudaTensor(2, 4, 5)
tmp = res:clone()
integralCUDA(t, res, tmp)
print(res) -- :view(5, 2*4)

(1,.,.) = 
   1   2   3   4
   5   6   7   8
   9  10  11  12

(2,.,.) = 
   1   2   3   4
   5   6   7   8
   9  10  11  12
[torch.CudaTensor of size 2x3x4]



(1,.,.) = 
   0   0   0   0   0
   0   1   3   6  10
   0   6  14  24  36
   0  15  33  54  78

(2,.,.) = 
   0   0   0   0   0
   0   1   3   6  10
   0   6  14  24  36
   0  15  33  54  78
[torch.CudaTensor of size 2x4x5]

